<a href="https://colab.research.google.com/github/IhkafAmeny/ABIDE-Dataset/blob/main/Downloading_ABIDE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nilearn --quiet
from nilearn import datasets
import pandas as pd
import os
import zipfile
from google.colab import files

print("Downloading ABIDE dataset (CPAC pipeline)")
abide = datasets.fetch_abide_pcp(
    n_subjects=250,
    pipeline='cpac',
    derivatives=['func_preproc'],
    quality_checked=True
)
print("Download complete.")

image_paths = abide.func_preproc[:1000]
phenotypes = abide.phenotypic

df = pd.DataFrame({
    'image_path': image_paths,
    'subject_id': phenotypes['SUB_ID'],
    'diagnosis': phenotypes['DX_GROUP'].map({1: 'Control', 2: 'Autism'}),
    'sex': phenotypes['SEX'].map({1: 'Male', 2: 'Female'}),
    'age': phenotypes['AGE_AT_SCAN'],
    'site': phenotypes['SITE_ID']
})

csv_path = "/content/phenotypes.csv"
df.to_csv(csv_path, index=False)
print(f"CSV saved at: {csv_path}")

zip_path = "/content/ABIDE_250.zip"
with zipfile.ZipFile(zip_path, 'w') as zipf:
    print("Compressing ABIDE files...")
    for path in df['image_path']:
        arcname = os.path.basename(path)
        zipf.write(path, arcname=arcname)
    zipf.write(csv_path, arcname="phenotypes.csv")
print(f"Compression complete: {zip_path}")

files.download(zip_path)